# LOAD LIBRARIES

In [ ]:
import sys
sys.path.append("../")
print(sys.path)
import os
import copy
import numpy as np
import pandas as pd
import random
import config
import matplotlib.pyplot as plt
from datetime import datetime

if not os.path.exists(config.NUMPY_DIR):
    os.makedirs(config.NUMPY_DIR)
if not os.path.exists(os.path.join(config.RESULT_DIR)):
    os.makedirs(os.path.join(config.RESULT_DIR))

print(config.NUMPY_DIR)

# MERGE DATA

In [ ]:
wv_dir = os.path.join(config.DATA_DIR, "basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2/basin_mean_forcing/nldas")
sf_dir = os.path.join(config.DATA_DIR, "basin_timeseries_v1p2_metForcing_obsFlow/basin_dataset_public_v1p2/usgs_streamflow")

dir_list = sorted(set(os.listdir(wv_dir)).intersection(os.listdir(sf_dir)))

for directory in dir_list:
    print(directory)
    wv_file_list = [file for file in sorted(os.listdir(os.path.join(wv_dir, directory))) if file.endswith(".txt")]
    sf_file_list = [file for file in sorted(os.listdir(os.path.join(sf_dir, directory))) if file.endswith(".txt")]    
    basin_list = sorted(set([file.split("_")[0] for file in wv_file_list]).intersection([file.split("_")[0] for file in sf_file_list]))
    
    for basin in basin_list:
        wv_file = os.path.join(wv_dir, directory, "{}_lump_nldas_forcing_leap.txt".format(basin))
        sf_file = os.path.join(sf_dir, directory, "{}_streamflow_qc.txt".format(basin))
        
        with open(wv_file, 'r') as fp:
            content = fp.readlines()
            area = int(content[2])

        wv_df = pd.read_csv(wv_file, sep='\s+', header=3)
        wv_dates = (wv_df.Year.map(str) + "/" + wv_df.Mnth.map(str) + "/" + wv_df.Day.map(str))
        wv_df.index = pd.to_datetime(wv_dates, format="%Y/%m/%d")
        
        sf_df = pd.read_csv(sf_file, sep='\s+', header=None)
        sf_dates = (sf_df[1].map(str) + "/" + sf_df[2].map(str) + "/" + sf_df[3].map(str))
        sf_df.index = pd.to_datetime(sf_dates, format="%Y/%m/%d")

        df = wv_df.join(sf_df[4])
        df = df.rename(columns = {4:'SF'})
        df['SF'] = df['SF'].fillna(-999.0)
        sf_vals = df.SF
        nan_idx = sf_vals==-999
        sf_vals = 28316846.592 * sf_vals * 86400 / (area * 10**6)
        sf_vals[nan_idx] = -999
        df.SF = sf_vals

        df = df[['PRCP(mm/day)', 'SRAD(W/m2)', 'Tmax(C)',  'Tmin(C)',  'Vp(Pa)', 'SF']]
        path = os.path.join(config.DATA_DIR, "RAW_DATA", "{}.txt".format(basin))
        df.to_csv(path, sep='\t', index_label="Date")

# SAVE DATA

In [ ]:
# Static Features
basin_attr = pd.read_csv(os.path.join(config.DATA_DIR, "RAW_DATA", "physic_27.csv"), dtype = {'gauge_id': object})

basin_df = pd.read_csv(os.path.join(config.DATA_DIR, "RAW_DATA", "{}.txt".format(basin_attr.gauge_id.values[0])), sep='\t', index_col=0)

columns = list(basin_attr.columns.values[1:]) + list(basin_df.columns.values)
date = np.array([datetime.strptime(day, '%Y-%m-%d').date() for day in basin_df.index.values])

data = np.zeros((len(basin_attr), basin_df.shape[0], len(columns)))
for i,basin in enumerate(basin_attr.gauge_id.values):
    basin_df = pd.read_csv(os.path.join(config.DATA_DIR, "RAW_DATA", "{}.txt".format(basin)), sep='\t', index_col=0)
    weather_drivers = basin_df.values
    static_features = basin_attr.iloc[i].values
    static_features = np.repeat(static_features[np.newaxis, :], weather_drivers.shape[0], axis=0)
    basin_data = np.concatenate((static_features[:,1:], weather_drivers), axis=1)
    data[i] = basin_data
print(data.shape)

np.save(os.path.join(config.RESULT_DIR, "Basin_List"), basin_attr.gauge_id.values)
# np.save(os.path.join(config.DATA_DIR, "RAW_DATA", "feature_names.npy"), columns)
# np.save(os.path.join(config.DATA_DIR, "RAW_DATA", "dates.npy"), date)
# np.save(os.path.join(config.DATA_DIR, "RAW_DATA", "data.npy"), data.astype(np.float32))

In [ ]:
Basin_list =np.load(os.path.join(config.RESULT_DIR, "Basin_List.npy"), allow_pickle=True)
Basin_list

# LOAD DATA & REMOVE LEAP YEAR

In [ ]:
data = np.load(os.path.join(config.DATA_DIR, "RAW_DATA", "data.npy")).astype(np.float32)
date = np.load(os.path.join(config.DATA_DIR, "RAW_DATA", "dates.npy"), allow_pickle=True)
print(date.shape, data.shape, "Original data")

date_df = pd.DataFrame(date)
date_df = pd.to_datetime(date_df[0], errors='coerce')
print(date_df.loc[date_df.dt.month == 2].loc[date_df.dt.day == 29])
date = np.delete(date, date_df.loc[date_df.dt.month == 2].loc[date_df.dt.day == 29].index.values)
data = np.delete(data, date_df.loc[date_df.dt.month == 2].loc[date_df.dt.day == 29].index.values, axis=1)
print(date.shape, data.shape, "Leap year removed data")

feature_names = np.load(os.path.join(config.DATA_DIR, "RAW_DATA", "feature_names.npy"), allow_pickle=True)

# NAN VALUES

In [ ]:
data[data==config.unknown] = np.nan
for feature,channel in enumerate(config.channels):
    print("#NaNs:{}\tChannel:{}".format(np.sum(np.isnan(data[:,:,feature])), feature_names[channel]))
data[np.isnan(data)] = config.unknown

In [ ]:
print(data.shape)
res = np.argwhere(data == 0)
res

# NORMALIZE DATA

In [ ]:
print("Original Feature stats")
data[data==config.unknown] = np.nan



for feature,channel in enumerate(config.channels):
    print("Mean:{:.4f}\tStddev:{:.4f}\tMin:{:.4f}\tMax:{:.4f}\tChannel:{}".format(np.nanmean(data[:,:,feature]), np.nanstd(data[:,:,feature]), np.nanmin(data[:,:,feature]), np.nanmax(data[:,:,feature]), feature_names[channel]))
data[np.isnan(data)] = config.unknown
# mean_real = np.nanmean(data, axis=(0,1))
# print(mean_real)
# std_real = np.nanstd(data, axis=(0,1))
# np.save(os.path.join(config.RESULT_DIR, "Mean__original_data"), mean_real)
# np.save(os.path.join(config.RESULT_DIR, "std_original_data"), std_real)

# LOG TRANSFORM OF STREAMFLOWS
normalized_data = data.copy()
normalized_data[:,:,-1][normalized_data[:,:,-1]!=config.unknown] = np.log10(config.add+normalized_data[:,:,-1][normalized_data[:,:,-1]!=config.unknown])

print("Log Transform Feature stats")
normalized_data[normalized_data==config.unknown] = np.nan
for feature,channel in enumerate(config.channels):
    print("Mean:{:.4f}\tStddev:{:.4f}\tMin:{:.4f}\tMax:{:.4f}\tChannel:{}".format(np.nanmean(normalized_data[:,:,feature]), np.nanstd(normalized_data[:,:,feature]), np.nanmin(normalized_data[:,:,feature]), np.nanmax(normalized_data[:,:,feature]), feature_names[channel]))
normalized_data[np.isnan(normalized_data)] = config.unknown

# Z NORMALIZE DATA
normalized_data[normalized_data==config.unknown] = np.nan
mean = np.reshape(np.nanmean(normalized_data, axis=(0,1)), (1,1,-1))
std = np.reshape(np.nanstd(normalized_data, axis=(0,1)), (1,1,-1))
np.save(os.path.join(config.RESULT_DIR, "Mean__original_data"), mean)
np.save(os.path.join(config.RESULT_DIR, "std_original_data"), std)

normalized_data = (normalized_data-mean)/std
normalized_data[np.isnan(normalized_data)] = config.unknown

print("Normalized Feature stats")
normalized_data[normalized_data==config.unknown] = np.nan
for feature,channel in enumerate(config.channels):
    print("Mean:{:.4f}\tStddev:{:.4f}\tMin:{:.4f}\tMax:{:.4f}\tChannel:{}".format(np.nanmean(normalized_data[:,:,feature]), np.nanstd(normalized_data[:,:,feature]), np.nanmin(normalized_data[:,:,feature]), np.nanmax(normalized_data[:,:,feature]), feature_names[channel]))
normalized_data[np.isnan(normalized_data)] = config.unknown
normalized_data[np.isnan(normalized_data)].shape

In [ ]:
normalized_data.shape

In [ ]:
# import seaborn as sns

In [ ]:
# plt.figure(figsize=(9, 9))   
# plt.hist(data[:,0,9], color = 'blue', edgecolor = 'black',bins = 50)
# plt.savefig(os.path.join(config.RESULT_DIR,"carbonate_rocks_distribution.png"))

In [ ]:
# plt.figure(figsize=(9, 9))   
# #plt.hist(data[:,0,9], color = 'blue', edgecolor = 'black',bins = 100)
# sns.kdeplot(data[:,0,9])
# plt.savefig(os.path.join(config.RESULT_DIR,"carbonate_rocks_distribution.png"))

In [ ]:
# Generate extra layer
layer = np.repeat(np.arange(1, 532)[:,None], normalized_data.shape[1], axis=1)
print(layer.shape)
# Get dimensions right...
layer = np.expand_dims(layer, axis=2)

# ... and finally append to data
normalized_data = np.append(normalized_data, layer, axis=2)
normalized_data.shape
normalized_data[:,0,33]

# CREATE TRAIN DATA

In [ ]:
# number_of_rows = normalized_data.shape[0]

# random_indices = np.random.choice(number_of_rows, size=500, replace=False)

# normalized_data = normalized_data[random_indices,:]
# normalized_data.shape

np.random.seed(4)
np.random.shuffle(normalized_data)
normalized_data.shape


In [ ]:
normalized_data[:,0,33]

In [ ]:
basin_order = normalized_data[:,0,33].astype(int).tolist()
basin_order = [x - 1 for x in basin_order]
Basin_list[basin_order]
import pickle
with open(os.path.join(config.NUMPY_DIR, "basin_list_ealstm"), "wb") as fp:   #pickling
    basin_list = pickle.dump(Basin_list[basin_order],fp)

In [ ]:
with open(os.path.join(config.NUMPY_DIR, "basin_list_ealstm"), "rb") as fp:   #pickling
    basin_list = pickle.load(fp)
basin_list    

In [ ]:
# 'train_start': pd.to_datetime('01101999', format='%d%m%Y'),
# 'train_end': pd.to_datetime('30092008', format='%d%m%Y'),
# 'val_start': pd.to_datetime('01101989', format='%d%m%Y'),
# 'val_end': pd.to_datetime('30091999', format='%d%m%Y')

In [ ]:

train_date = date[np.where(date==pd.Timestamp(2001, 10,1, 0))[0][0]:np.where(date==pd.Timestamp(2008, 9, 30, 0))[0][0]+1]
train_data = normalized_data[:,np.where(date==pd.Timestamp(2001, 10,1, 0))[0][0]:np.where(date==pd.Timestamp(2008, 9, 30, 0))[0][0]+1]
strided_train_data = np.zeros((train_data.shape[0], 2*(len(train_date)//config.window), config.window, train_data.shape[-1]))
print(train_date.shape, train_data.shape, strided_train_data.shape)

i = 0
k = 0
while i<len(train_date):
    strided_train_data[:, k] = train_data[:, i:i+config.window]
    k += 1

    stride = config.window//2
    if strided_train_data[:, k].shape == train_data[:, i+stride:i+stride+config.window].shape:
        strided_train_data[:, k] = train_data[:, i+stride:i+stride+config.window]
        k += 1

    i = i+config.window
#strided_train_data = strided_train_data[:300, :k]
strided_train_data = strided_train_data[:, :k]
print("Train:{}".format(strided_train_data.shape))

# CREATE VALIDATION DATA

In [ ]:
validation_date = date[np.where(date==pd.Timestamp(1999, 10,1, 0))[0][0]:np.where(date==pd.Timestamp(2001, 9, 30, 0))[0][0]+1]
validation_data = normalized_data[:,np.where(date==pd.Timestamp(1999, 10,1, 0))[0][0]:np.where(date==pd.Timestamp(2001, 9, 30, 0))[0][0]+1]
strided_validation_data = np.zeros((validation_data.shape[0], 2*(len(validation_date)//config.window), config.window, validation_data.shape[-1]))
print(validation_date.shape, validation_data.shape, strided_validation_data.shape)

i = 0
k = 0
while i<len(validation_date):
    strided_validation_data[:, k] = validation_data[:, i:i+config.window]
    k += 1

    stride = config.window//2
    if strided_validation_data[:, k].shape == validation_data[:, i+stride:i+stride+config.window].shape:
        strided_validation_data[:, k] = validation_data[:, i+stride:i+stride+config.window]
        k += 1

    i = i+config.window
#strided_validation_data = strided_validation_data[:300, :k]
strided_validation_data = strided_validation_data[:, :k]
print("validation:{}".format(strided_validation_data.shape))

# CREATE TEST DATA

In [ ]:
test_date = date[np.where(date==pd.Timestamp(1989, 10,1, 0))[0][0]:np.where(date==pd.Timestamp(1999, 9, 30, 0))[0][0]+1]

test_data = normalized_data[:,np.where(date==pd.Timestamp(1989, 10,1, 0))[0][0]:np.where(date==pd.Timestamp(1999, 9, 30, 0))[0][0]+1]
print(test_data.shape)
strided_test_data = np.zeros((test_data.shape[0], 2*(len(test_date)//config.window), config.window, test_data.shape[-1]))
print(test_date.shape, test_data.shape, strided_test_data.shape)

i = 0
k = 0
while i<len(test_date):
    strided_test_data[:, k] = test_data[:, i:i+config.window]
    k += 1

    stride = config.window//2
    if strided_test_data[:, k].shape == test_data[:, i+stride:i+stride+config.window].shape:
        strided_test_data[:, k] = test_data[:, i+stride:i+stride+config.window]
        k += 1

    i = i+config.window
#strided_test_data = strided_test_data[300:, :k]
strided_test_data = strided_test_data[:, :k]
print("test:{}".format(strided_test_data.shape))

# SAVE DATA

In [ ]:
print("Data:{}\tTrain_Basin".format(strided_train_data.shape))
np.save(os.path.join(config.NUMPY_DIR, "train_data_basin_ealstm"), strided_train_data.astype(np.float32))
print("Data:{}\tValidation_Basin".format(strided_validation_data.shape))
np.save(os.path.join(config.NUMPY_DIR, "validation_data_basin_ealstm"), strided_validation_data.astype(np.float32))
print("Data:{}\tTest_Basin".format(strided_test_data.shape))
np.save(os.path.join(config.NUMPY_DIR, "test_data_basin_ealstm"), strided_test_data.astype(np.float32))
# print("Data:{}\tTrain_hidden_Basin".format(strided_hidden_train_data.shape))
# np.save(os.path.join(config.NUMPY_DIR, "train_hidden_data_basin"), strided_hidden_train_data.astype(np.float32))

# AVERAGE STREAMFLOWS

In [ ]:
node_flow = np.zeros((data.shape[0], 3))
for node in range(data.shape[0]):
    node_labels = data[node,:,-1]
    mean = np.mean(node_labels[node_labels!=config.unknown])
    median = np.median(node_labels[node_labels!=config.unknown])
    node_flow[node,0] = node
    node_flow[node,1] = mean
    node_flow[node,2] = median

indices = np.argsort(node_flow[:,1])
node_flow = node_flow[indices]

print("Min Obs:{:.4f}\tNode:{}".format(node_flow[0,1], node_flow[0,0]))
print("Max Obs:{:.4f}\tNode:{}".format(node_flow[-1,1], node_flow[-1,0]))

print("Plot Average Stream Flows")
rows = (node_flow.shape[0]//100)+1
fig = plt.figure(figsize=(20, 5*rows))
fontsize = 10
width = 0.4
for row in range(rows):
    start = 100*row
    end = 100*(row+1)
    ax = fig.add_subplot(rows,1,row+1)
    ax.set_xticks(range(len(node_flow[start:end,0])))
    ax.set_xticklabels(node_flow[start:end,0], fontsize=fontsize, rotation=90)
    ax.bar(np.array(range(len(node_flow[start:end,0]))), node_flow[start:end,1], width=width , color="Green", label = "Mean")
    ax.bar(np.array(range(len(node_flow[start:end,0]))), node_flow[start:end,2], width=width , color="Blue", label = "Median")
    ax.legend(loc="upper left")
fig.tight_layout(pad=0)
plt.savefig(os.path.join(config.RESULT_DIR, "Average_Flows_basin.png"), format = "png")
plt.show()

In [ ]:
node = 312
feature = 31
window = config.window

i=0
while i < len(data[node,:,-1]):
    fig = plt.figure(figsize=(30,3))
    fontsize = 20
    ax = fig.add_subplot(111)
    flow = data[node,:,-1]
    flow[flow==config.unknown] = np.nan
    sf = ax.plot(flow[i:i+window], color="Blue", label = "Streamflow")
    lns = sf
    
    if feature is not None:
        ax2 = ax.twinx()
        feat = ax2.plot(data[node,:,feature][i:i+window], color="orange", label = "Feature:{}".format(feature), alpha = 0.8)
        lns = sf+feat
    
    labs = [l.get_label() for l in lns]
    ax.legend(lns, labs, loc="upper right", fontsize=fontsize)
    ax.set_xticks(range(0, window, 5))
    ax.set_xticklabels(date[i:i+window][::5], rotation=90)
    fig.tight_layout(pad=0)
    plt.show()
    plt.close()
    i=i+window